In [2]:
#import data
import pandas as pd
import numpy as np
import importlib
import glob
import numpy as np 

#change paths
arr = pd.read_csv("test/transfer_plate.csv").iloc[:, 1:-4].to_numpy()
transfer_plate_labels=pd.read_csv("test/transfer_plate.csv").iloc[:, -4:].to_numpy()
test_data = pd.read_csv("test/96_samples.csv",header=None).iloc[:, 1:].to_numpy()

csv_files = glob.glob("train/*.csv")
dataframes = {}
for file in csv_files:

    dataframes[file] = pd.read_csv(file)



In [3]:
transfer_plate_labels_1=[]
for i in range(len(transfer_plate_labels)-1):

    transfer_plate_labels_1.append(transfer_plate_labels[i])
    transfer_plate_labels_1.append(transfer_plate_labels[i])

transfer_plate_labels_1=np.array(transfer_plate_labels_1)[:192,-3:]
transfer_plate_labels_1=np.array(transfer_plate_labels_1,dtype=np.float32)


In [ ]:
from scipy.interpolate import interp1d

target_wavenumbers = np.linspace(300, 1942, 1340)
interpolated = []
conc=[]

for file in csv_files:
    # Convert column names to float

    wavelengths = dataframes[file].columns[:-5].astype(float)
    # Get only the spectral data
    intensities_y = (dataframes[file].values)
    
    conc.append(intensities_y[:,-5:-2])
    #normalization steps --1-- scaling

    intensities_y=intensities_y/(np.max(intensities_y))



    # Interpolate each row (spectrum)

    for row in intensities_y:

        f = interp1d(wavelengths, row[:-5], kind='quadratic', fill_value='extrapolate')

        interpolated.append(f(target_wavenumbers))



interpolated=np.array(interpolated)

#normalization step --2--

interpolated=(interpolated-np.mean(interpolated))/np.std(interpolated)
conc=np.concatenate(conc)



0.9999999999999993

In [17]:
arr = np.char.strip(arr.astype(str), "[]").astype(float)
test_data = np.char.strip(test_data.astype(str),"[]").astype(float)

# normalization step --- 1 --- scaling
arr=arr/np.max(arr)

test_data=test_data/np.max(test_data)


In [18]:
wavelengths=np.linspace(65,3350,2048)#make sure this is the actual interval.

#interpolate test_data and the transfer_plate datasets(same format)
transfer_plate=[]
clean_test=[]
for row in arr:

    f = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')
    transfer_plate.append(f(target_wavenumbers))

for row in test_data:

    g = interp1d(wavelengths, row, kind='quadratic', fill_value='extrapolate')

    clean_test.append(g(target_wavenumbers))


transfer_plate=(transfer_plate-np.mean(transfer_plate))/np.std(transfer_plate)
clean_test=(clean_test-np.mean(clean_test))/np.std(clean_test)


transfer_plate=np.array(transfer_plate,dtype=np.float32)

In [26]:
import Data_augment as Da
importlib.reload(Da)

train_data=np.concatenate((transfer_plate,interpolated))
labels=np.concatenate((transfer_plate_labels_1,conc))
labels_scaled,labels_max= Da.reversible_scaler(labels,None)

labels_max

0


array([16.7344 ,  3.05065,  3.94541])

In [27]:
from Raman_Dataset import RamanDataset
from torch.utils.data import DataLoader
import Data_augment as Da
importlib.reload(Da)



dataset1 = RamanDataset(train_data, labels_scaled)




loader = DataLoader(dataset1, batch_size=32, shuffle=True)

# Example: iterate
for spectra_batch, targets_batch in loader:
    print(spectra_batch.shape)  # (batch_size, 1, num_points)

    print(targets_batch.shape)  # (batch_size, 3)
    break

0
0
torch.Size([32, 1, 1340])
torch.Size([32, 3])


-------Big Model training ------

In [ ]:
import SpectraCNN as CNN
import torch
import torch.nn as nn
importlib.reload(CNN)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


model = CNN.SpectraCNN()

model = model.to(device) #move to gpu if available

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

epochs=100

for epoch in range(epochs):
    running_loss=0
    print(epoch)
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch}: Loss {running_loss/len(loader)}")

In [28]:
clean_test=np.array(clean_test,dtype=np.float32)

In [29]:
test_loader = DataLoader(RamanDataset(clean_test,None, train=False), batch_size=32, shuffle=False)


model.eval()  # switch to evaluation mode
predictions = []

with torch.no_grad():
    for spectra in test_loader:
        outputs = model(spectra)  # shape (batch, 3)
        predictions.append(outputs)

predictions = torch.cat(predictions).numpy()
predictions2=Da.reversible_scaler(predictions,labels_max,reverse=True)

NameError: name 'model' is not defined

Cross validation for model tuning and ensembling

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import KFold
from SpectraCNN import SpectraCNN


import Cross_validation as cv
importlib.reload(cv)

train_loss, val_loss = cv.run_kfold_cv(train_data, labels_scaled, k=5, batch_size=32, epochs=250, model_class = SpectraCNN)


---- Fold 1/5 ----
Epoch 1/250 - Val Loss: 0.0644
Epoch 2/250 - Val Loss: 0.0587
Epoch 3/250 - Val Loss: 0.0483
Epoch 4/250 - Val Loss: 0.0446
Epoch 5/250 - Val Loss: 0.0476
Epoch 6/250 - Val Loss: 0.0424
Epoch 7/250 - Val Loss: 0.0411
Epoch 8/250 - Val Loss: 0.0415
Epoch 9/250 - Val Loss: 0.0409
Epoch 10/250 - Val Loss: 0.0408
Epoch 11/250 - Val Loss: 0.0385
Epoch 12/250 - Val Loss: 0.0386
Epoch 13/250 - Val Loss: 0.0390
Epoch 14/250 - Val Loss: 0.0382
Epoch 15/250 - Val Loss: 0.0374
Epoch 16/250 - Val Loss: 0.0364
Epoch 17/250 - Val Loss: 0.0362
Epoch 18/250 - Val Loss: 0.0367
Epoch 19/250 - Val Loss: 0.0369
Epoch 20/250 - Val Loss: 0.0385
Epoch 21/250 - Val Loss: 0.0352
Epoch 22/250 - Val Loss: 0.0359
Epoch 23/250 - Val Loss: 0.0346
Epoch 24/250 - Val Loss: 0.0339
Epoch 25/250 - Val Loss: 0.0346
Epoch 26/250 - Val Loss: 0.0330
Epoch 27/250 - Val Loss: 0.0317
Epoch 28/250 - Val Loss: 0.0348
Epoch 29/250 - Val Loss: 0.0353
Epoch 30/250 - Val Loss: 0.0355
Epoch 31/250 - Val Loss: 0.03

KeyboardInterrupt: 

In [ ]:
# Load models
models = []
for i in range(1, 6):
    model = SpectraCNN()
    model.load_state_dict(torch.load(f"C:\\Users\\joaof\\Downloads\\model{i}.pth", map_location="cpu"))
    model.eval()
    models.append(model)

clean_test=np.array(clean_test,dtype=np.float32)
test_loader = DataLoader(RamanDataset(clean_test,None, train=False), batch_size=32, shuffle=False)

# Inference with ensemble averaging
all_outputs = []
with torch.no_grad():
    for (x) in test_loader:

        preds = [m(x) for m in models]                 # list of [batch_size, output_dim]
        
        mean_pred = torch.stack(preds).mean(dim=0)     # average over models
        
        all_outputs.append(mean_pred)
        
final_results = torch.cat(all_outputs)  # [num_samples, output_dim]

final_results_descaled=Da.reversible_scaler(final_results,max=labels_max,reverse=True)

#load predictions to sample_submission.csv

template= pd.read_csv("test/sample_submission.csv")


averaged = final_results_descaled.reshape(-1, 2, 3).mean(axis=1)

template[['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']] = averaged

# Save to new CSV
template.to_csv("test/sample_submission.csv", index=False)

(1, 32, 3)
(2, 32, 3)
(3, 32, 3)
(4, 32, 3)
(5, 32, 3)
(6, 32, 3)
torch.Size([192, 3])
